In [138]:
# Cell 1: Imports & parser setup
import sympy as sp
from sympy import sin, cos, exp, log, E
import mpmath as mp
from sympy.parsing.sympy_parser import (
    parse_expr,
    standard_transformations,
    implicit_multiplication_application,
)

x = sp.symbols('x')
_transformations = standard_transformations + (implicit_multiplication_application,)

In [139]:
# Cell 2: Manual method detection

def detect_methods(expr, raw_str=None):
    steps = []
    # 0) Linearity
    if expr.is_Add:
        steps.append("Linearity: separate sum/difference")
    # Detect term methods
    terms = expr.args if expr.is_Add else (expr,)
    for term in terms:
        steps += _detect_term_methods(term)
    # remove duplicates preserving order
    return list(dict.fromkeys(steps))


def _detect_term_methods(term):
    t = []
    num, den = term.as_numer_denom()

    # 1) Constant multiple rule: factor out numeric coefficient
    if term.is_Mul:
        coeff, _ = term.as_coeff_mul(x)
        if coeff not in (1, -1):
            t.append("Constant multiple rule")

    # 2) Log power rule: ln(x^n) -> n ln(x)
    for atom in term.atoms(log):
        arg = atom.args[0]
        if arg.is_Pow and arg.base == x:
            t.append("Log power rule: ln(x^n)=n ln(x)")

    # 3) Natural-log rule: ∫1/x or ln(x) use log integration
    if term == 1/x or (term.has(log) and not any(r.startswith("Log power rule") for r in t)):
        t.append("Natural-log rule")

    # 4) Integration by parts: single log or product of log/exp with other
    if term.has(log) and not term.is_Add:
        t.append("Integration by parts")
    elif term.is_Mul and den == 1:
        factors = term.as_ordered_factors()
        has_le = any(f.has(log) or f.has(exp) for f in factors)
        has_other = any(not (f.has(log) or f.has(exp)) for f in factors)
        if has_le and has_other:
            t.append("Integration by parts")

    # 5) Reverse power rule: ∫x^n
    if (term.is_Pow and term.base == x) or (term.is_polynomial(x) and not term.has(log) and not term.has(exp)):
        t.append("Reverse power rule")

    # 6) Trigonometric integration
    if term.has(sp.sin) or term.has(sp.cos) or term.has(sp.tan):
        t.append("Trigonometric integration")

    # 7) Partial fractions: rational functions
    if den.is_Add:
        t.append("Partial fractions")
        if any(arg.has(log) for arg in den.args):
            t.append("Inverse-log pattern (u = ln(x))")
    elif den != 1 and num.is_polynomial(x) and den.is_polynomial(x):
        t.append("Partial fractions")

    # 8) U-substitution: non-polynomial compositions except log/exp
    if "Integration by parts" not in t and (
        (den != 1 and num.is_polynomial(x) and den.is_polynomial(x))
        or term.has(sp.sin) or term.has(sp.cos)
        or (term.has(sp.exp) and not term.has(log))
    ):
        t.append("U-substitution")

    return t

In [140]:
# Cell 3:: Integral computation
def compute_integral(expr, a, b):
    # Attempt symbolic antiderivative
    F = sp.integrate(expr, x)
    if not F.has(sp.Integral):
        Fx = sp.simplify(F)
        res = Fx.subs(x, b) - Fx.subs(x, a)
        try:
            return float(sp.N(res))
        except Exception:
            pass
    # Fallback numeric in case of sadness
    f_num = sp.lambdify(x, expr, "mpmath")
    return float(mp.quad(f_num, [a, b]))

In [141]:
# Cell 4: Print steps in order
def print_ordered_methods(expr, raw_str=None):
    PRIORITY = [
        "Linearity: separate sum/difference",
        "Log power rule: ln(x^n)=n ln(x)",
        "Constant multiple rule",
        "Natural-log rule",
        "Integration by parts",
        "Reverse power rule",
        "Trigonometric integration",
        "Partial fractions",
        "Inverse-log pattern (u = ln(x))",
        "U-substitution",
    ]
    raw = detect_methods(expr, raw_str)
    for step in PRIORITY:
        if step in raw:
            print(f"- {step}")

In [143]:
# Cell 5: Interactive prompt
def predict_interactive():
    f_str = input("Function (e.g. x^2+3x^2 or 4*sqrt(x)+ln(x)): ")
    a = float(input("Lower bound: "))
    b = float(input("Upper bound: "))
    print(f"\n∫[{f_str}] dx from {a} to {b}:")

    expr = parse_expr(
        f_str.replace("^", "**").replace("ln(", "log("),
        local_dict={"sqrt": sp.sqrt, "log": sp.log, "ln": sp.log,
                    "exp": sp.exp, "e": sp.E},
        transformations=_transformations,
        evaluate=False
    )

    print_ordered_methods(expr, f_str)
    result = compute_integral(expr, a, b)
    print(f"Result = {result}")

# Steps that can be printed
    return {"function": f_str, "steps": [s for s in [
                "Linearity: separate sum/difference", "Natural-log rule",
                "Reverse power rule", "Integration by parts",
                "Trigonometric integration", "Partial fractions",
                "Inverse-log pattern (u = ln(x))", "U-substitution"
            ] if s in detect_methods(expr, f_str)],
            "result": result}

# Calling user-input
if __name__ == "__main__":
    outcome = predict_interactive()


∫[cos(2x)+5x^4] dx from 2.0 to 9.0:
- Linearity: separate sum/difference
- Constant multiple rule
- Reverse power rule
- Trigonometric integration
- U-substitution
Result = 59017.002907624264
